# appunti hdf5

formato per scambio di dati autodescrittivi 

autodescrittivo: data set, eg array n dimensionale di numeri, ha metadati addizionali associati ad esso che descrivono cose come rango 
	dell'array, numero di elementi in ogni dimensione, etc
	
prende principi da programmazione orientata agli ogg

array n-dim, immagini e tabelle possono esser immagazzinati nello stesso file ma in oggetti diversi

utente può capire contenuti del file a cui accede nei termini dei vari tipi di dati oggetto hdf5, segue descriz tipi dati oggetto

## hdf5 file struct

file hdf5 = directory + collection di data obj

ogni data obj ha un entry nella directory contenente puntatore a locazione del data obj, e informazione sul datatype

++ su datatypes: https://portal.hdfgroup.org/display/HDF5/Datatype+Basics

ci sono solo due data obj fondamentali in hdf5: groups e namespaces

file hdf5 sono organizzati gerarchicamente con due strutture primarie: groups, datasets


In [2]:
#= non va
# grafo creato con ipytree
# installazione : dal prompt di comandi di anaconda: conda install -c conda-forge ipytree
using PyCall
py"""
def pytree():
    from ipytree import Tree, Node
    tree=Tree()

    tree.add_node(Node("Node1"))
    tree.add_node(Node("Node2"))
    tree.add_node(Node("Node3"))

    tree
"""
mytree= py"pytree"
mytree()=# 

## gruppi

un gruppo hdf5 è una struttura contenente 0 o + oggetti

ogni gruppo contiene 2 parti: 

    . header: contiene nome gruppo, lista di attributi del gruppo
    
    . tabella simboli: lista tutti gli oggetti hdf5 appartenenti al gruppo
    
gerarchia gruppi/membri è simile a directory/file e path in UNIX:

/ <- indica gruppo root

/foo <- indica membro foo del gruppo root

/foo/loo <- indica membro loo del gruppo foo, il gruppo foo è membro del gruppo root
    
## dataset

dataset sono array multidimensionali di dati con metadati annessi, un dataset è immagazzinato in due parti: header e data array

### dataset header

contiene informazioni necessarie all interpretazione della porzione array del dataset e metadati(o puntatori a metadati)  che descrivano o annotino il dataset.

informazioni nell'header includono: nome dell oggetto, dimensioni oggetto, number-type dell oggetto



++ hdf5 vs hdf-eos5?